# Best Practices for Prompt Engineering

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api


In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_type = "azure"
openai.api_version = "2023-06-01-preview"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

# 1. 최신 모델 사용

최상의 결과를 얻으려면 최신 모델을 사용하십시오.

# 2. Prompt는 역할과 컨텍스트를 구분하여 작성
Task는 System Role에 컨택스트를 넣고, 사용자 명령은 User Role에 컨택스트를 넣으세요. 경우에 따라서 구분자를 사용하세요. (``` """ ''')

In [5]:
content='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress.'
        
prompt=f"""Summarize the text below as a bullet point list of the most important points. \n\n \
    ```{content}```
    """
            
messages=[
    {"role": "user", "content": prompt},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

- OpenAI and Microsoft are extending their partnership.
- The investment from Microsoft is multi-year and multi-billion dollar.
- The partnership will allow OpenAI to continue independent research and develop safer, more useful, and powerful AI.
- OpenAI remains a capped-profit company and is governed by the OpenAI non-profit.
- This structure allows OpenAI to raise capital without sacrificing its core beliefs on sharing benefits and prioritizing safety.
- Microsoft shares OpenAI's vision and values.
- The partnership is instrumental to OpenAI's progress.


In [8]:
system_message='Summarize the text below as a bullet point list of the most important points.'
user_message='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\n \
            In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
            capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
            raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
            broadly sharing benefits and the need to prioritize safety. \
            Microsoft shares this vision and our values, and our partnership is instrumental to our progress.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

- OpenAI and Microsoft are extending their partnership
- Microsoft has made a multi-year, multi-billion dollar investment in OpenAI
- The investment will allow OpenAI to continue independent research and develop AI that is safe, useful, and powerful
- OpenAI remains a capped-profit company and is governed by the OpenAI non-profit in pursuit of their mission to ensure advanced AI benefits all of humanity
- This structure allows OpenAI to raise capital without sacrificing their core beliefs about sharing benefits and prioritizing safety
- Microsoft shares OpenAI's vision and values, and their partnership is instrumental to OpenAI's progress


# 3. 원하는 컨텍스트, 결과, 길이, 형식, 스타일 등에 대해 구체적이고 가능한 한 자세히 설명하십시오.

In [10]:
prompt='Write a poem about OpenAI.'

response = openai.ChatCompletion.create(
            engine=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
            n=1
        )

print(response['choices'][0]["message"]["content"])

In the world of technology and AI,
Where machines are learning to fly,
There's an entity that stands out,
For making AI accessible, no doubt.

OpenAI, the name echoes loud,
A champion of AI, amongst the crowd,
With its cutting-edge research and innovations,
It's paving the way for new explorations.

From language models to neural nets,
OpenAI's breakthroughs we cannot forget,
GPT-3 was a game-changer of its own kind,
A language model like no one could find.

But innovation isn't all OpenAI represents,
It's a champion of ethics, that never relents,
The idea of safe and responsible AI,
The quest that OpenAI will never let die.

With the world of AI constantly changing,
OpenAI's work is always engaging,
As it strives for betterment every day,
For a brighter and smarter AI way.

So here's to OpenAI, the champion of AI,
For leading us towards a future, way up high,
May its research and innovations never cease,
As they continue to bring groundbreaking peace.


In [11]:
prompt='Write a short inspiring poem about OpenAI, \
        focusing on the recent DALL-E product launch in the style of Ernest Hemingway'

response = openai.ChatCompletion.create(
            engine=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
        )

print(response['choices'][0]["message"]["content"])

OpenAI, bold and true,
Recent launch of DALL-E, homage to you.
Creativity unleashed,
Innovative tech has been reached.

Inspiration found in every line,
As DALL-E prompts us to realign,
Our vision of what's possible,
With OpenAI, we feel unstoppable.

With every step, we surge ahead,
Exploring new limits, breaking new bread.
Boldness and courage, our guiding force,
OpenAI, we salute to your course.

For DALL-E is the foundation,
Of an endless source of creation.
An ode to the future, we stand so bright,
With OpenAI, we achieve new heights.


# 4. 예제를 통해 원하는 출력 형식을 설명하십시오 (예 1, 예 2).

In [12]:
system_message='Extract the companyn names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34}'
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
                
response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        max_tokens=400,
        n=1
)

print(response['choices'][0]["message"]["content"])

{"text": "OpenAI", "start": 28, "end": 34}
{"text": "Microsoft", "start": 50, "end": 59}
{"text": "2019", "start": 99, "end": 102}
{"text": "2021", "start": 107, "end": 110}


In [17]:
system_message='Extract the entities mentioned in the text below. \
    Extract the important entities mentioned in the text below. \
    First extract all company names, then extract all years, \
    then extract specific topics which fit the content and finally extract general overarching themes\n\n \
    Desired format: \
    Company names: <comma_separated_list_of_company_names> \
    Years: -||- \
    Specific topics: -||- \
    General themes: -||- '
    
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
    This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
    in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
    increasingly safe, useful, and powerful.'

# system_message='아래 텍스트에 언급 된 엔티티를 추출하십시오. \
#     아래 텍스트에 언급 된 중요한 엔티티를 추출하십시오. \
#     먼저 모든 회사 이름을 추출한 다음 몇 년 동안 추출하고 \
#     그런 다음 내용에 맞는 특정 주제를 추출하고 마지막으로 일반적인 가장 중요한 테마를 추출하십시오.\n\n \
#     Desired format: \
#     회사 이름: <comma_separated_list_of_company_names> \
#     연도: -||- \
#     특정 주제: -||- \
#     일반적인 테마: -||- '

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

회사 이름: OpenAI, Microsoft
연도: 2019, 2021
특정 주제: 인공지능, 안전, 유용성, 강력함
일반적인 테마: 기술협력, 연구 및 개발


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune 

In [18]:
# zero-shot

system_message='Extract keywords from the below text.'
user_message  ='We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful.'

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


In [19]:
# few-shot

messages=[
    {"role": "system", "content": 'Extract keywords from the corresponding texts below.'},
    {"role": "user", "content": 'Text 1: Stripe provides APIs that web developers can use to integrate payment processing into their websites and mobile applications.'},
    {"role": "assistant", "content": 'Keywords 1: Stripe, payment processing, APIs, web developers, websites, mobile applications'},
    {"role": "user", "content": 'Text 2: OpenAI has trained cutting-edge language models that are very good at understanding and generating text. Our API provides access to these models and can be used to solve virtually any task that involves processing language.'},
    {"role": "assistant", "content": 'Keywords 2: OpenAI, language models, text processing, API.'},
    {"role": "user", "content": 'Text 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])


Keywords 3: OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful.


# 6. 애매한 표현과 부정확한 설명을 줄입니다

In [21]:
# 애매하고 부정확한 설명

messages=[
    {"role": "system", "content": 'Write a description for a new product.'},
    {"role": "user", "content": 'This product is a new generation of car seat. The description for this product should be fairly short, a few sentences only, and not too much more.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

Introducing the revolutionary new car seat that offers unparalleled safety and comfort for your child. This seat features advanced impact protection and is designed to fit a wide range of vehicles, making it the perfect choice for families on the go. With its sleek, modern design and easy-to-clean fabrics, this car seat will become your go-to solution for safe and comfortable travel.


In [22]:
# 명확한 설명

messages=[
    {"role": "system", "content": 'Write a description for a new product.'},
    {"role": "user", "content": 'This product is a new generation of car seat. Use a 3 to 5 sentence paragraph to describe this product.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

Introducing the state-of-the-art car seat! This revolutionary product is designed to prioritize the safety and comfort of your child while they are seated in your vehicle. Made from premium quality materials, this car seat boasts adjustable settings that allow it to grow with your child, making it a long-lasting investment. This seat is engineered to absorb energy in the event of a collision, ensuring maximum protection for your child at all times. Additionally, this car seat is easy to install and can be adjusted to different recline angles depending on the preference of your child.


# 7. 하지 말아야 할 것을 말하지 말고 대신 해야 할 것을 말하십시오.

In [23]:
# 하지 말아야 할 것에 대한 정의 없을 경우 발생하는 사건

messages=[
    {"role": "system", "content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT.'},
    {"role": "user", "content": 'I can’t log in to my account.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

I'm sorry to hear that. Have you tried resetting your password?


In [25]:
# 해야 하는 것에 대한 정의를 하는 경우

messages=[
    {"role": "system", "content": 'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq'},
    {"role": "user", "content": 'I can’t log in to my account.'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
    n=1
)

print(response['choices'][0]["message"]["content"])

I'm sorry about that! Let's see if we can figure out the issue. Have you tried resetting your password? Sometimes that can help if you're having trouble logging in. You can find instructions on how to reset your password in our help article here: www.samplewebsite.com/help/faq.


# 8. 코드 생성 - 모델을 특정 패턴으로 시작할 수 있도록 "시작하는 단어"를 사용하세요.

In [26]:
messages=[
    {"role": "user", "content": '# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers'},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

def miles_to_km():
    mile = float(input("Enter the distance in miles: "))
    km = mile * 1.60934
    print(mile, "miles is equal to", km, "kilometers")


In [5]:
# text-davinchi-003 or GPT-4 모델에서 결과물이 더 잘 나옵니다.

messages=[
    {"role": "user", "content": '# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\n\
                import '},
]

response = openai.ChatCompletion.create(
    engine=model,
    messages=messages,
    max_tokens=400,
)

print(response['choices'][0]["message"]["content"])

# text-davinchi-003 에서 테스트한 결과 입니다.
# response = openai.Completion.create(
#             engine=os.getenv('DEPLOYMENT_NAME_003'),
#             prompt='# Write a simple python function that \n\
#                 # 1. Ask me for a number in mile\n\
#                 # 2. It converts miles to kilometers\n\n\
#                 import ',
#             max_tokens=400,
#         )

# print(response['choices'][0]['text'])

### output ###
# math
                
#                 def miles_to_km(miles):
#                     km = miles * 1.609
#                     return math.floor(km)
                    
#                 miles = float(input('Enter a value in miles:'))
#                 km = miles_to_km(miles)
#                 print(f'{miles} miles in kilometers is {km}km')

 math
                
                def miles_to_km(miles):
                    km = miles * 1.609
                    return math.floor(km)
                    
                miles = float(input('Enter a value in miles:'))
                km = miles_to_km(miles)
                print(f'{miles} miles in kilometers is {km}km')
